---

## 1. Introduction

In the previous notebooks, we've focused on **phase-based** connectivity metrics like the Phase
Locking Value (PLV). These metrics capture the **synchronization of oscillation timing** between
signals — whether two brain regions oscillate in phase with each other.

However, there's another fundamental way that neural signals can be related: through their
**amplitude fluctuations**. Even if two signals have completely unrelated phases, they might
show **co-fluctuation of oscillation strength** — when one signal's amplitude increases, so
does the other's.

This leads us to **amplitude-based connectivity metrics** like:
- **Envelope Correlation (CCorr)**: Pearson correlation between amplitude envelopes
- **Power Correlation (PowCorr)**: Correlation between instantaneous power values

**Key insight**: Amplitude fluctuations are **slower** than the oscillation itself. A 10 Hz
alpha oscillation might have envelope fluctuations at only 0.5-2 Hz. This means amplitude
coupling operates on a different timescale than phase coupling, potentially reflecting
different neural communication mechanisms.

In **hyperscanning**, amplitude coupling between participants may reflect shared arousal,
attention states, or cognitive engagement — even when their brain oscillations aren't
phase-locked.

---

## 2. What is the Amplitude Envelope?

The **amplitude envelope** is the curve that traces the peaks of an oscillating signal.
Think of it as the "outline" that hugs the oscillation from above and below.

For a **narrowband signal** (signal filtered to a specific frequency range), the envelope
represents the **instantaneous amplitude** — how strong the oscillation is at each moment.

**Mathematical definition**:
$$A(t) = |z(t)| = |x(t) + i \cdot \mathcal{H}\{x(t)\}|$$

Where:
- $z(t)$ is the analytic signal (from Hilbert transform)
- $|z(t)|$ is its magnitude = the envelope

**Key properties**:
- Envelope is always **positive** (it's a magnitude)
- Envelope changes **slowly** compared to the carrier oscillation
- Envelope captures **when** oscillations are strong vs weak

**Analogy**: AM radio uses amplitude modulation. The high-frequency carrier wave has an
envelope that carries the actual audio information. Similarly, brain oscillations have
envelopes that carry information about neural activity strength.

In [ ]:
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================
import sys
from pathlib import Path
from typing import Tuple

import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy.signal import hilbert, butter, filtfilt, welch
from scipy.ndimage import gaussian_filter1d
from scipy.stats import pearsonr

# Add src to path for local imports
sys.path.insert(0, str(Path.cwd().parents[2]))

from src.colors import COLORS
from src.filtering import bandpass_filter
from src.phase import compute_plv_simple

# Color aliases for convenience (using available COLORS keys)
PRIMARY_BLUE = COLORS["signal_1"]       # Sky Blue
PRIMARY_RED = COLORS["negative"]        # Coral Red
PRIMARY_GREEN = COLORS["signal_3"]      # Sage Green
SECONDARY_PURPLE = COLORS["signal_5"]   # Lavender
SECONDARY_ORANGE = COLORS["signal_4"]   # Golden
ACCENT_PURPLE = COLORS["high_sync"]     # Purple for accents
ACCENT_GOLD = COLORS["signal_4"]        # Golden for highlights

# Add envelope-specific colors to COLORS for this notebook
COLORS["signal"] = COLORS["signal_1"]
COLORS["envelope"] = COLORS["negative"]
COLORS["correlation"] = COLORS["high_sync"]

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# ============================================================================
# VISUALIZATION 1: Amplitude-Modulated Signal with Envelope
# ============================================================================

# Create an amplitude-modulated signal
fs = 500  # Sampling rate
duration = 3.0  # seconds
t = np.arange(0, duration, 1/fs)

# Carrier: 10 Hz oscillation
carrier_freq = 10  # Hz
carrier = np.sin(2 * np.pi * carrier_freq * t)

# Modulation: slow 0.5 Hz envelope variation
mod_freq = 0.5  # Hz
modulation = 0.5 + 0.5 * np.sin(2 * np.pi * mod_freq * t)  # Range [0, 1]

# AM signal
am_signal = modulation * carrier

# Extract envelope using Hilbert transform
analytic = hilbert(am_signal)
envelope = np.abs(analytic)

# Create figure
fig, ax = plt.subplots(figsize=(14, 5))

# Plot signal
ax.plot(t, am_signal, color=PRIMARY_BLUE, linewidth=0.8, alpha=0.7, label='AM Signal (10 Hz carrier)')

# Plot envelope (upper and lower)
ax.plot(t, envelope, color=PRIMARY_RED, linewidth=2.5, label='Envelope (upper)')
ax.plot(t, -envelope, color=PRIMARY_RED, linewidth=2.5, label='Envelope (lower)')

# Plot true modulation for comparison
ax.plot(t, modulation, color=PRIMARY_GREEN, linewidth=2, linestyle='--', 
        alpha=0.8, label='True modulation')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.set_title('Visualization 1: Amplitude Envelope "Hugs" the Oscillation', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.set_xlim(0, 3)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Carrier frequency: {carrier_freq} Hz")
print(f"Modulation frequency: {mod_freq} Hz")
print(f"→ The envelope captures the slow amplitude changes!")

In [ ]:
# ============================================================================
# VISUALIZATION 2: EEG-like Alpha Signal with Natural Envelope
# ============================================================================

# Generate realistic EEG-like signal with alpha activity
np.random.seed(42)
fs_eeg = 250
duration_eeg = 5.0
t_eeg = np.arange(0, duration_eeg, 1/fs_eeg)

# Create alpha bursts with varying amplitude
# Base alpha oscillation
alpha_freq = 10  # Hz

# Natural amplitude modulation (irregular bursts)
burst_mod = np.zeros(len(t_eeg))
# Add several bursts at random times
burst_times = [0.5, 1.5, 2.8, 4.0]  # seconds
burst_widths = [0.4, 0.6, 0.5, 0.7]  # seconds
burst_amps = [1.0, 0.7, 1.2, 0.9]

for bt, bw, ba in zip(burst_times, burst_widths, burst_amps):
    burst_mod += ba * np.exp(-((t_eeg - bt) ** 2) / (2 * (bw/3) ** 2))

# Add baseline and noise
burst_mod = 0.2 + burst_mod  # baseline

# Create alpha signal
alpha_signal = burst_mod * np.sin(2 * np.pi * alpha_freq * t_eeg)
alpha_signal += 0.15 * np.random.randn(len(t_eeg))  # Add noise

# Filter to alpha band and extract envelope
alpha_filtered = bandpass_filter(alpha_signal, 8, 13, fs_eeg)
alpha_analytic = hilbert(alpha_filtered)
alpha_envelope = np.abs(alpha_analytic)

# Create figure
fig, ax = plt.subplots(figsize=(14, 5))

# Plot filtered alpha signal
ax.plot(t_eeg, alpha_filtered, color=PRIMARY_BLUE, linewidth=0.8, 
        alpha=0.7, label='Alpha-filtered signal (8-13 Hz)')

# Plot envelope
ax.fill_between(t_eeg, -alpha_envelope, alpha_envelope, 
                color=PRIMARY_RED, alpha=0.3, label='Envelope region')
ax.plot(t_eeg, alpha_envelope, color=PRIMARY_RED, linewidth=2.5, label='Envelope')
ax.plot(t_eeg, -alpha_envelope, color=PRIMARY_RED, linewidth=2.5)

# Mark bursts
for i, bt in enumerate(burst_times):
    ax.axvline(x=bt, color='gray', linestyle=':', alpha=0.5)
    ax.annotate(f'Burst {i+1}', xy=(bt, alpha_envelope.max() * 1.1), 
                ha='center', fontsize=9, color='gray')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Amplitude (a.u.)', fontsize=12)
ax.set_title('Visualization 2: Alpha "Bursts" — Envelope Captures Activity Strength', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.set_xlim(0, 5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("→ The envelope reveals WHEN alpha activity is strong (bursts) vs weak")
print("→ This is crucial for understanding neural dynamics!")

---

## 3. Envelope vs Amplitude vs Power

These terms are often confused in the literature. Let's clarify:

| Term | Definition | Range | Units (if signal in µV) |
|------|------------|-------|-------------------------|
| **Signal amplitude** | $x(t)$ | $(-\infty, +\infty)$ | µV |
| **Instantaneous amplitude** | $|z(t)|$ | $[0, +\infty)$ | µV |
| **Envelope** | $|z(t)|$ | $[0, +\infty)$ | µV |
| **Instantaneous power** | $|z(t)|^2$ | $[0, +\infty)$ | µV² |

**Key distinctions**:
- **Envelope = Instantaneous amplitude** — these are synonymous!
- **Power = Envelope²** — squaring emphasizes large amplitudes
- **Band power** (from A03) is the *integral* of PSD over a frequency range

**When to use which?**
- **Envelope**: Standard for amplitude connectivity (CCorr)
- **Power**: Sometimes used, emphasizes peaks more
- In practice, results are often similar, but power is more sensitive to outliers

In [ ]:
# ============================================================================
# VISUALIZATION 3: Signal vs Envelope vs Power
# ============================================================================

# Use the alpha signal from before
power = alpha_envelope ** 2

# Create figure with 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(14, 9), sharex=True)

# Subplot 1: Original signal
axes[0].plot(t_eeg, alpha_filtered, color=PRIMARY_BLUE, linewidth=0.8)
axes[0].set_ylabel('Signal x(t)\n(µV)', fontsize=11)
axes[0].set_title('Original Signal: Can be positive or negative', 
                  fontsize=12, fontweight='bold')
axes[0].axhline(y=0, color='gray', linestyle='-', alpha=0.5)
axes[0].grid(True, alpha=0.3)

# Subplot 2: Envelope
axes[1].plot(t_eeg, alpha_envelope, color=PRIMARY_RED, linewidth=2)
axes[1].fill_between(t_eeg, 0, alpha_envelope, color=PRIMARY_RED, alpha=0.3)
axes[1].set_ylabel('Envelope |z(t)|\n(µV)', fontsize=11)
axes[1].set_title('Envelope (Instantaneous Amplitude): Always positive', 
                  fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Subplot 3: Power
axes[2].plot(t_eeg, power, color=ACCENT_PURPLE, linewidth=2)
axes[2].fill_between(t_eeg, 0, power, color=ACCENT_PURPLE, alpha=0.3)
axes[2].set_ylabel('Power |z(t)|²\n(µV²)', fontsize=11)
axes[2].set_xlabel('Time (s)', fontsize=12)
axes[2].set_title('Instantaneous Power: Squared envelope — peaks are emphasized', 
                  fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Visualization 3: Signal vs Envelope vs Power', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Compute correlation between envelope and power
corr_env_pow = np.corrcoef(alpha_envelope, power)[0, 1]
print(f"Correlation between envelope and power: {corr_env_pow:.4f}")
print("→ They are highly correlated, but power emphasizes peaks more!")

---

## 4. Extracting the Envelope — Complete Pipeline

The standard workflow for envelope extraction:

1. **Filter** the signal to the frequency band of interest
2. Apply the **Hilbert transform** to get the analytic signal
3. Take the **magnitude** to get the envelope

**Why filtering first?**
- The envelope is only meaningful for **narrowband** signals
- A broadband signal has no single "envelope"
- The frequency band determines which oscillation we track

**Different bands = different envelopes = different dynamics!**

In [ ]:
# ============================================================================
# FUNCTION 1: extract_envelope
# ============================================================================

def extract_envelope(
    signal: NDArray[np.floating],
    fs: float,
    band: Tuple[float, float],
    filter_order: int = 4
) -> NDArray[np.floating]:
    """
    Extract the amplitude envelope from a signal for a specific frequency band.
    
    Parameters
    ----------
    signal : NDArray[np.floating]
        Input signal.
    fs : float
        Sampling frequency in Hz.
    band : Tuple[float, float]
        Frequency band as (low_freq, high_freq) in Hz.
    filter_order : int, optional
        Order of the bandpass filter (default: 4).
    
    Returns
    -------
    NDArray[np.floating]
        Amplitude envelope of the band-filtered signal.
    
    Examples
    --------
    >>> envelope = extract_envelope(eeg_signal, fs=250, band=(8, 13))
    """
    # Step 1: Bandpass filter
    filtered = bandpass_filter(signal, band[0], band[1], fs, order=filter_order)
    
    # Step 2: Hilbert transform
    analytic = hilbert(filtered)
    
    # Step 3: Magnitude = envelope
    envelope = np.abs(analytic)
    
    return envelope


# Test the function
test_envelope = extract_envelope(alpha_signal, fs_eeg, (8, 13))
print(f"Envelope extracted: {len(test_envelope)} samples")
print(f"Envelope range: [{test_envelope.min():.3f}, {test_envelope.max():.3f}]")

In [ ]:
# ============================================================================
# VISUALIZATION 4: Envelopes Across Different Frequency Bands
# ============================================================================

# Generate a broadband EEG-like signal with multiple components
np.random.seed(123)
fs_multi = 250
duration_multi = 6.0
t_multi = np.arange(0, duration_multi, 1/fs_multi)

# Create signal with theta, alpha, and beta components
# Each with different amplitude modulations
theta_mod = 0.5 + 0.5 * np.sin(2 * np.pi * 0.3 * t_multi)  # Slow modulation
alpha_mod = 0.3 + 0.7 * np.exp(-((t_multi - 2) ** 2) / 0.5) + 0.6 * np.exp(-((t_multi - 4.5) ** 2) / 0.3)
beta_mod = 0.4 + 0.4 * np.sin(2 * np.pi * 0.8 * t_multi + 1)  # Faster modulation

# Generate oscillations
theta = theta_mod * np.sin(2 * np.pi * 6 * t_multi)
alpha = alpha_mod * np.sin(2 * np.pi * 10 * t_multi)
beta = beta_mod * np.sin(2 * np.pi * 20 * t_multi)

# Combine with noise
broadband = theta + alpha + 0.5 * beta + 0.3 * np.random.randn(len(t_multi))

# Extract envelopes for each band
env_theta = extract_envelope(broadband, fs_multi, (4, 8))
env_alpha = extract_envelope(broadband, fs_multi, (8, 13))
env_beta = extract_envelope(broadband, fs_multi, (13, 30))

# Create figure
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Raw signal
axes[0].plot(t_multi, broadband, color='gray', linewidth=0.5, alpha=0.7)
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Raw Broadband Signal (Contains Multiple Rhythms)', 
                  fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Theta envelope
theta_filt = bandpass_filter(broadband, 4, 8, fs_multi)
axes[1].plot(t_multi, theta_filt, color=COLORS["theta"], linewidth=0.5, alpha=0.5)
axes[1].plot(t_multi, env_theta, color=COLORS["theta"], linewidth=2.5, label='Theta envelope')
axes[1].plot(t_multi, -env_theta, color=COLORS["theta"], linewidth=2.5)
axes[1].set_ylabel('Theta (4-8 Hz)', fontsize=11)
axes[1].set_title('Theta Band: Slow, Sustained Modulation', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Alpha envelope
alpha_filt = bandpass_filter(broadband, 8, 13, fs_multi)
axes[2].plot(t_multi, alpha_filt, color=COLORS["alpha"], linewidth=0.5, alpha=0.5)
axes[2].plot(t_multi, env_alpha, color=COLORS["alpha"], linewidth=2.5, label='Alpha envelope')
axes[2].plot(t_multi, -env_alpha, color=COLORS["alpha"], linewidth=2.5)
axes[2].set_ylabel('Alpha (8-13 Hz)', fontsize=11)
axes[2].set_title('Alpha Band: Clear Burst Pattern', fontsize=12, fontweight='bold')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)

# Beta envelope
beta_filt = bandpass_filter(broadband, 13, 30, fs_multi)
axes[3].plot(t_multi, beta_filt, color=COLORS["beta"], linewidth=0.5, alpha=0.5)
axes[3].plot(t_multi, env_beta, color=COLORS["beta"], linewidth=2.5, label='Beta envelope')
axes[3].plot(t_multi, -env_beta, color=COLORS["beta"], linewidth=2.5)
axes[3].set_ylabel('Beta (13-30 Hz)', fontsize=11)
axes[3].set_xlabel('Time (s)', fontsize=12)
axes[3].set_title('Beta Band: Faster Envelope Fluctuations', fontsize=12, fontweight='bold')
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)

plt.suptitle('Visualization 4: Different Bands Have Different Envelope Dynamics', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("→ Each frequency band has its own characteristic envelope dynamics")
print("→ The choice of band determines what neural activity we track!")

---

## 5. Temporal Dynamics of the Envelope

A crucial insight: **envelopes fluctuate on slower timescales** than the oscillation itself.

- A 10 Hz alpha oscillation completes 10 cycles per second
- But its envelope might only change significantly over 0.5-2 seconds
- The envelope is essentially a **low-frequency signal**

This has important implications:
- Envelope dynamics reflect **neural modulation processes**
- They can reveal cognitive states, attention, arousal
- For connectivity, envelope correlation captures **slow co-fluctuation**

Let's verify this by looking at the frequency content of an envelope:

In [ ]:
# ============================================================================
# FUNCTION 2: compute_envelope_psd
# ============================================================================

def compute_envelope_psd(
    envelope: NDArray[np.floating],
    fs: float
) -> Tuple[NDArray[np.floating], NDArray[np.floating]]:
    """
    Compute the Power Spectral Density of an envelope signal.
    
    Parameters
    ----------
    envelope : NDArray[np.floating]
        Amplitude envelope time series.
    fs : float
        Sampling frequency in Hz.
    
    Returns
    -------
    Tuple[NDArray[np.floating], NDArray[np.floating]]
        (frequencies, psd) arrays.
    
    Notes
    -----
    Useful for analyzing envelope dynamics — confirms that envelopes
    contain primarily low-frequency content.
    """
    from scipy.signal import welch
    
    # Use Welch's method with appropriate parameters for slow signals
    nperseg = min(len(envelope), int(fs * 2))  # 2-second windows
    frequencies, psd = welch(envelope, fs=fs, nperseg=nperseg)
    
    return frequencies, psd

In [ ]:
# ============================================================================
# VISUALIZATION 5: Envelope is a Slow Signal
# ============================================================================

# Use alpha envelope from previous example
freqs_env, psd_env = compute_envelope_psd(env_alpha, fs_multi)

# Also compute PSD of the filtered signal for comparison
from scipy.signal import welch
freqs_sig, psd_sig = welch(alpha_filt, fs=fs_multi, nperseg=int(fs_multi * 2))

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Top left: Alpha signal and envelope in time domain
axes[0, 0].plot(t_multi, alpha_filt, color=COLORS["alpha"], linewidth=0.5, 
                alpha=0.5, label='Alpha signal')
axes[0, 0].plot(t_multi, env_alpha, color=PRIMARY_RED, linewidth=2, label='Envelope')
axes[0, 0].set_xlabel('Time (s)', fontsize=11)
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('Time Domain: Alpha Signal & Envelope', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)

# Top right: Zoomed view
zoom_start, zoom_end = 1.5, 2.5
zoom_mask = (t_multi >= zoom_start) & (t_multi <= zoom_end)
axes[0, 1].plot(t_multi[zoom_mask], alpha_filt[zoom_mask], color=COLORS["alpha"], 
                linewidth=1, alpha=0.7, label='Alpha signal')
axes[0, 1].plot(t_multi[zoom_mask], env_alpha[zoom_mask], color=PRIMARY_RED, 
                linewidth=2.5, label='Envelope')
axes[0, 1].set_xlabel('Time (s)', fontsize=11)
axes[0, 1].set_ylabel('Amplitude', fontsize=11)
axes[0, 1].set_title('Zoomed: Signal Oscillates Fast, Envelope Changes Slowly', 
                     fontsize=12, fontweight='bold')
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: PSD of the alpha signal
axes[1, 0].semilogy(freqs_sig, psd_sig, color=COLORS["alpha"], linewidth=2)
axes[1, 0].axvspan(8, 13, color=COLORS["alpha"], alpha=0.2, label='Alpha band')
axes[1, 0].set_xlabel('Frequency (Hz)', fontsize=11)
axes[1, 0].set_ylabel('PSD (log scale)', fontsize=11)
axes[1, 0].set_title('PSD of Alpha Signal: Peak at 10 Hz', fontsize=12, fontweight='bold')
axes[1, 0].set_xlim(0, 50)
axes[1, 0].legend(loc='upper right')
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: PSD of the envelope
axes[1, 1].semilogy(freqs_env, psd_env, color=PRIMARY_RED, linewidth=2)
axes[1, 1].axvspan(0, 2, color=PRIMARY_RED, alpha=0.2, label='< 2 Hz')
axes[1, 1].set_xlabel('Frequency (Hz)', fontsize=11)
axes[1, 1].set_ylabel('PSD (log scale)', fontsize=11)
axes[1, 1].set_title('PSD of Envelope: Power Concentrated < 2 Hz!', 
                     fontsize=12, fontweight='bold', color=PRIMARY_RED)
axes[1, 1].set_xlim(0, 10)
axes[1, 1].legend(loc='upper right')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Visualization 5: The Envelope is a SLOW Signal', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("→ The alpha SIGNAL oscillates at ~10 Hz")
print("→ The alpha ENVELOPE fluctuates at < 2 Hz")
print("→ Envelope dynamics operate on a different (slower) timescale!")

---

## 6. Envelope Smoothing

Raw envelopes can be **noisy**, especially with:
- Short data segments
- Low signal-to-noise ratio (SNR)
- High-frequency noise leaking through

**Smoothing** reduces rapid fluctuations and reveals the underlying trend.

**Common methods**:
1. **Moving average**: Simple, but distorts edges
2. **Gaussian smoothing**: Better edge behavior
3. **Low-pass filtering**: Most principled approach

**Trade-off**: More smoothing = less temporal resolution

In [ ]:
# ============================================================================
# FUNCTIONS 3, 4, 5: Envelope Smoothing Methods
# ============================================================================

def smooth_envelope_moving_average(
    envelope: NDArray[np.floating],
    window_samples: int
) -> NDArray[np.floating]:
    """
    Smooth envelope using a simple moving average.
    
    Parameters
    ----------
    envelope : NDArray[np.floating]
        Amplitude envelope time series.
    window_samples : int
        Width of the moving average window in samples.
    
    Returns
    -------
    NDArray[np.floating]
        Smoothed envelope.
    """
    kernel = np.ones(window_samples) / window_samples
    return np.convolve(envelope, kernel, mode='same')


def smooth_envelope_gaussian(
    envelope: NDArray[np.floating],
    sigma_samples: float
) -> NDArray[np.floating]:
    """
    Smooth envelope using Gaussian filtering.
    
    Parameters
    ----------
    envelope : NDArray[np.floating]
        Amplitude envelope time series.
    sigma_samples : float
        Standard deviation of the Gaussian kernel in samples.
    
    Returns
    -------
    NDArray[np.floating]
        Smoothed envelope.
    """
    return gaussian_filter1d(envelope, sigma=sigma_samples)


def smooth_envelope_lowpass(
    envelope: NDArray[np.floating],
    fs: float,
    cutoff: float = 2.0
) -> NDArray[np.floating]:
    """
    Smooth envelope by low-pass filtering.
    
    Parameters
    ----------
    envelope : NDArray[np.floating]
        Amplitude envelope time series.
    fs : float
        Sampling frequency in Hz.
    cutoff : float, optional
        Low-pass cutoff frequency in Hz (default: 2.0).
    
    Returns
    -------
    NDArray[np.floating]
        Smoothed envelope.
    """
    from scipy.signal import butter, filtfilt
    
    # Design low-pass filter
    nyq = fs / 2
    normalized_cutoff = cutoff / nyq
    b, a = butter(4, normalized_cutoff, btype='low')
    
    # Apply zero-phase filtering
    return filtfilt(b, a, envelope)

In [ ]:
# ============================================================================
# VISUALIZATION 6: Comparing Smoothing Methods
# ============================================================================

# Create a noisy envelope for demonstration
np.random.seed(456)
fs_smooth = 250
duration_smooth = 4.0
t_smooth = np.arange(0, duration_smooth, 1/fs_smooth)

# True underlying envelope (smooth)
true_envelope = 0.5 + 0.3 * np.sin(2 * np.pi * 0.5 * t_smooth)
true_envelope += 0.4 * np.exp(-((t_smooth - 2) ** 2) / 0.3)

# Add noise to create "raw" envelope
raw_envelope = true_envelope + 0.15 * np.random.randn(len(t_smooth))
raw_envelope = np.maximum(raw_envelope, 0)  # Envelope must be positive

# Apply smoothing methods
window_ms = 50  # 50 ms window
window_samples = int(window_ms * fs_smooth / 1000)

sigma_ms = 20  # 20 ms sigma
sigma_samples = sigma_ms * fs_smooth / 1000

smooth_ma = smooth_envelope_moving_average(raw_envelope, window_samples)
smooth_gauss = smooth_envelope_gaussian(raw_envelope, sigma_samples)
smooth_lp = smooth_envelope_lowpass(raw_envelope, fs_smooth, cutoff=2.0)

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top left: Raw vs True
axes[0, 0].plot(t_smooth, true_envelope, color='black', linewidth=2, 
                linestyle='--', label='True envelope')
axes[0, 0].plot(t_smooth, raw_envelope, color='gray', linewidth=1, 
                alpha=0.7, label='Noisy envelope')
axes[0, 0].set_xlabel('Time (s)', fontsize=11)
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('Raw Noisy Envelope vs True', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)

# Top right: Moving average
axes[0, 1].plot(t_smooth, true_envelope, color='black', linewidth=2, 
                linestyle='--', label='True', alpha=0.5)
axes[0, 1].plot(t_smooth, raw_envelope, color='gray', linewidth=0.5, alpha=0.3)
axes[0, 1].plot(t_smooth, smooth_ma, color=PRIMARY_BLUE, linewidth=2, 
                label=f'Moving avg ({window_ms} ms)')
axes[0, 1].set_xlabel('Time (s)', fontsize=11)
axes[0, 1].set_ylabel('Amplitude', fontsize=11)
axes[0, 1].set_title('Moving Average Smoothing', fontsize=12, fontweight='bold')
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: Gaussian
axes[1, 0].plot(t_smooth, true_envelope, color='black', linewidth=2, 
                linestyle='--', label='True', alpha=0.5)
axes[1, 0].plot(t_smooth, raw_envelope, color='gray', linewidth=0.5, alpha=0.3)
axes[1, 0].plot(t_smooth, smooth_gauss, color=PRIMARY_GREEN, linewidth=2, 
                label=f'Gaussian (σ={sigma_ms} ms)')
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_ylabel('Amplitude', fontsize=11)
axes[1, 0].set_title('Gaussian Smoothing', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc='upper right')
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: Low-pass
axes[1, 1].plot(t_smooth, true_envelope, color='black', linewidth=2, 
                linestyle='--', label='True', alpha=0.5)
axes[1, 1].plot(t_smooth, raw_envelope, color='gray', linewidth=0.5, alpha=0.3)
axes[1, 1].plot(t_smooth, smooth_lp, color=ACCENT_PURPLE, linewidth=2, 
                label='Low-pass (2 Hz)')
axes[1, 1].set_xlabel('Time (s)', fontsize=11)
axes[1, 1].set_ylabel('Amplitude', fontsize=11)
axes[1, 1].set_title('Low-Pass Filtering', fontsize=12, fontweight='bold')
axes[1, 1].legend(loc='upper right')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Visualization 6: Envelope Smoothing Methods', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Compare errors
mse_raw = np.mean((raw_envelope - true_envelope) ** 2)
mse_ma = np.mean((smooth_ma - true_envelope) ** 2)
mse_gauss = np.mean((smooth_gauss - true_envelope) ** 2)
mse_lp = np.mean((smooth_lp - true_envelope) ** 2)

print("Mean Squared Error from true envelope:")
print(f"  Raw:            {mse_raw:.4f}")
print(f"  Moving average: {mse_ma:.4f}")
print(f"  Gaussian:       {mse_gauss:.4f}")
print(f"  Low-pass:       {mse_lp:.4f}")
print("\n→ All smoothing methods improve the estimate!")

---

## 7. Envelope Correlation — Preview of Connectivity

Two signals can have **correlated amplitude fluctuations** even if their phases are unrelated.

**Envelope correlation** measures this amplitude coupling:
- Compute envelope for each signal
- Calculate Pearson correlation between the two envelopes

**Interpretation**: When signal 1 is "strong", is signal 2 also "strong"?

This is **fundamentally different** from phase synchronization:
- PLV measures phase coupling (timing alignment)
- Envelope correlation measures amplitude coupling (power co-fluctuation)

Both can occur independently or together!

In [ ]:
# ============================================================================
# FUNCTION 6: compute_envelope_correlation
# ============================================================================

def compute_envelope_correlation(
    signal1: NDArray[np.floating],
    signal2: NDArray[np.floating],
    fs: float,
    band: Tuple[float, float]
) -> float:
    """
    Compute envelope correlation between two signals.
    
    Parameters
    ----------
    signal1 : NDArray[np.floating]
        First input signal.
    signal2 : NDArray[np.floating]
        Second input signal.
    fs : float
        Sampling frequency in Hz.
    band : Tuple[float, float]
        Frequency band as (low_freq, high_freq) in Hz.
    
    Returns
    -------
    float
        Pearson correlation coefficient between envelopes.
    """
    # Extract envelopes
    env1 = extract_envelope(signal1, fs, band)
    env2 = extract_envelope(signal2, fs, band)
    
    # Compute Pearson correlation
    corr, _ = pearsonr(env1, env2)
    
    return corr

In [ ]:
# ============================================================================
# VISUALIZATION 7: Correlated Envelopes
# ============================================================================

# Create two signals with CORRELATED envelopes but INDEPENDENT phases
np.random.seed(789)
fs_corr = 250
duration_corr = 6.0
t_corr = np.arange(0, duration_corr, 1/fs_corr)

# Shared envelope modulation
shared_modulation = 0.5 + 0.5 * np.sin(2 * np.pi * 0.4 * t_corr)
shared_modulation += 0.3 * np.exp(-((t_corr - 3) ** 2) / 0.5)

# Signal 1: alpha with shared modulation
phase1 = 2 * np.pi * 10 * t_corr + np.random.uniform(0, 2*np.pi)
signal1_corr = shared_modulation * np.sin(phase1) + 0.1 * np.random.randn(len(t_corr))

# Signal 2: alpha with shared modulation but DIFFERENT phase
phase2 = 2 * np.pi * 10 * t_corr + np.random.uniform(0, 2*np.pi)  # Independent phase
signal2_corr = shared_modulation * np.sin(phase2) + 0.1 * np.random.randn(len(t_corr))

# Extract envelopes
env1_corr = extract_envelope(signal1_corr, fs_corr, (8, 13))
env2_corr = extract_envelope(signal2_corr, fs_corr, (8, 13))

# Compute correlation
env_correlation = np.corrcoef(env1_corr, env2_corr)[0, 1]

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top left: Signal 1 with envelope
axes[0, 0].plot(t_corr, bandpass_filter(signal1_corr, 8, 13, fs_corr), 
                color=PRIMARY_BLUE, linewidth=0.5, alpha=0.5)
axes[0, 0].plot(t_corr, env1_corr, color=PRIMARY_BLUE, linewidth=2, label='Envelope 1')
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('Signal 1 (Alpha)', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)

# Top right: Signal 2 with envelope
axes[0, 1].plot(t_corr, bandpass_filter(signal2_corr, 8, 13, fs_corr), 
                color=PRIMARY_RED, linewidth=0.5, alpha=0.5)
axes[0, 1].plot(t_corr, env2_corr, color=PRIMARY_RED, linewidth=2, label='Envelope 2')
axes[0, 1].set_ylabel('Amplitude', fontsize=11)
axes[0, 1].set_title('Signal 2 (Alpha)', fontsize=12, fontweight='bold')
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: Both envelopes overlaid
axes[1, 0].plot(t_corr, env1_corr, color=PRIMARY_BLUE, linewidth=2, label='Envelope 1')
axes[1, 0].plot(t_corr, env2_corr, color=PRIMARY_RED, linewidth=2, label='Envelope 2')
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_ylabel('Amplitude', fontsize=11)
axes[1, 0].set_title('Envelopes Overlaid: They Co-Fluctuate!', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc='upper right')
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: Scatter plot
axes[1, 1].scatter(env1_corr, env2_corr, color=ACCENT_PURPLE, alpha=0.3, s=20)
# Add regression line
z = np.polyfit(env1_corr, env2_corr, 1)
p = np.poly1d(z)
env1_sorted = np.sort(env1_corr)
axes[1, 1].plot(env1_sorted, p(env1_sorted), color='black', linewidth=2, linestyle='--')
axes[1, 1].set_xlabel('Envelope 1', fontsize=11)
axes[1, 1].set_ylabel('Envelope 2', fontsize=11)
axes[1, 1].set_title(f'Envelope Correlation: r = {env_correlation:.3f}', 
                     fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Visualization 7: Amplitude Coupling (Correlated Envelopes)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Envelope correlation: {env_correlation:.3f}")
print("→ High correlation means amplitudes co-fluctuate!")
print("→ Note: Phases were independent — this is pure amplitude coupling")

In [ ]:
# ============================================================================
# VISUALIZATION 8: Uncorrelated Envelopes
# ============================================================================

# Create two signals with INDEPENDENT envelopes
np.random.seed(321)

# Signal 1: its own modulation pattern
mod1 = 0.5 + 0.5 * np.sin(2 * np.pi * 0.3 * t_corr)
signal1_indep = mod1 * np.sin(2 * np.pi * 10 * t_corr) + 0.1 * np.random.randn(len(t_corr))

# Signal 2: different modulation pattern
mod2 = 0.5 + 0.5 * np.sin(2 * np.pi * 0.5 * t_corr + np.pi)  # Different freq & phase
mod2 += 0.3 * np.exp(-((t_corr - 1.5) ** 2) / 0.3)  # Burst at different time
signal2_indep = mod2 * np.sin(2 * np.pi * 10 * t_corr) + 0.1 * np.random.randn(len(t_corr))

# Extract envelopes
env1_indep = extract_envelope(signal1_indep, fs_corr, (8, 13))
env2_indep = extract_envelope(signal2_indep, fs_corr, (8, 13))

# Compute correlation
env_correlation_indep = np.corrcoef(env1_indep, env2_indep)[0, 1]

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top left: Signal 1 with envelope
axes[0, 0].plot(t_corr, bandpass_filter(signal1_indep, 8, 13, fs_corr), 
                color=PRIMARY_BLUE, linewidth=0.5, alpha=0.5)
axes[0, 0].plot(t_corr, env1_indep, color=PRIMARY_BLUE, linewidth=2, label='Envelope 1')
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('Signal 1 (Alpha)', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)

# Top right: Signal 2 with envelope
axes[0, 1].plot(t_corr, bandpass_filter(signal2_indep, 8, 13, fs_corr), 
                color=PRIMARY_RED, linewidth=0.5, alpha=0.5)
axes[0, 1].plot(t_corr, env2_indep, color=PRIMARY_RED, linewidth=2, label='Envelope 2')
axes[0, 1].set_ylabel('Amplitude', fontsize=11)
axes[0, 1].set_title('Signal 2 (Alpha)', fontsize=12, fontweight='bold')
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: Both envelopes overlaid
axes[1, 0].plot(t_corr, env1_indep, color=PRIMARY_BLUE, linewidth=2, label='Envelope 1')
axes[1, 0].plot(t_corr, env2_indep, color=PRIMARY_RED, linewidth=2, label='Envelope 2')
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_ylabel('Amplitude', fontsize=11)
axes[1, 0].set_title('Envelopes Overlaid: No Co-Fluctuation', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc='upper right')
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: Scatter plot
axes[1, 1].scatter(env1_indep, env2_indep, color=ACCENT_PURPLE, alpha=0.3, s=20)
axes[1, 1].set_xlabel('Envelope 1', fontsize=11)
axes[1, 1].set_ylabel('Envelope 2', fontsize=11)
axes[1, 1].set_title(f'Envelope Correlation: r = {env_correlation_indep:.3f}', 
                     fontsize=12, fontweight='bold', color=PRIMARY_RED)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Visualization 8: No Amplitude Coupling (Uncorrelated Envelopes)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Envelope correlation: {env_correlation_indep:.3f}")
print("→ Low correlation means amplitudes fluctuate independently")
print("→ No amplitude coupling between these signals")

---

## 8. The Volume Conduction Problem for Amplitude

Volume conduction affects amplitude metrics too!

If two EEG channels pick up the **same underlying source**, their envelopes will be 
highly correlated — but this is **spurious correlation**, not true connectivity.

**Solutions** (briefly mentioned here, detailed in later notebooks):
- **Orthogonalization**: Remove the shared signal component before correlation
- **Source localization**: Analyze in source space rather than sensor space
- **Laplacian reference**: Spatial filtering to reduce volume conduction

This is why imaginary coherence (F02) and orthogonalized amplitude correlation (H01)
are preferred in many connectivity analyses.

In [ ]:
# ============================================================================
# VISUALIZATION 9: Volume Conduction Creates Spurious Envelope Correlation
# ============================================================================

# Simulate volume conduction: one source, two sensors
np.random.seed(555)
fs_vc = 250
duration_vc = 5.0
t_vc = np.arange(0, duration_vc, 1/fs_vc)

# True source signal with amplitude modulation
source_mod = 0.5 + 0.5 * np.sin(2 * np.pi * 0.5 * t_vc)
source_signal = source_mod * np.sin(2 * np.pi * 10 * t_vc)

# Two channels "pick up" the same source with different weights + independent noise
weight1, weight2 = 1.0, 0.7  # Different mixing weights
noise_level = 0.15

channel1 = weight1 * source_signal + noise_level * np.random.randn(len(t_vc))
channel2 = weight2 * source_signal + noise_level * np.random.randn(len(t_vc))

# Extract envelopes
env_ch1 = extract_envelope(channel1, fs_vc, (8, 13))
env_ch2 = extract_envelope(channel2, fs_vc, (8, 13))

# Compute correlation
spurious_corr = np.corrcoef(env_ch1, env_ch2)[0, 1]

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# Top left: Source signal
axes[0, 0].plot(t_vc, source_signal, color='black', linewidth=1, alpha=0.7, label='Source')
axes[0, 0].plot(t_vc, source_mod, color=PRIMARY_RED, linewidth=2, linestyle='--', 
                label='True envelope')
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('True Source Signal (One Neural Source)', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)

# Top right: Two channels
axes[0, 1].plot(t_vc, channel1, color=PRIMARY_BLUE, linewidth=0.5, alpha=0.5, label='Channel 1')
axes[0, 1].plot(t_vc, channel2 + 3, color=ACCENT_PURPLE, linewidth=0.5, alpha=0.5, 
                label='Channel 2 (offset)')
axes[0, 1].set_ylabel('Amplitude', fontsize=11)
axes[0, 1].set_title('Two EEG Channels: Same Source + Noise', fontsize=12, fontweight='bold')
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: Envelopes
axes[1, 0].plot(t_vc, env_ch1, color=PRIMARY_BLUE, linewidth=2, label='Envelope Ch1')
axes[1, 0].plot(t_vc, env_ch2, color=ACCENT_PURPLE, linewidth=2, label='Envelope Ch2')
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_ylabel('Amplitude', fontsize=11)
axes[1, 0].set_title('Envelopes are Highly Similar (Spuriously!)', 
                     fontsize=12, fontweight='bold', color=PRIMARY_RED)
axes[1, 0].legend(loc='upper right')
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: Warning
axes[1, 1].scatter(env_ch1, env_ch2, color=ACCENT_PURPLE, alpha=0.3, s=20)
z = np.polyfit(env_ch1, env_ch2, 1)
p = np.poly1d(z)
env1_sorted = np.sort(env_ch1)
axes[1, 1].plot(env1_sorted, p(env1_sorted), color=PRIMARY_RED, linewidth=3)
axes[1, 1].set_xlabel('Envelope Ch1', fontsize=11)
axes[1, 1].set_ylabel('Envelope Ch2', fontsize=11)
axes[1, 1].set_title(f'SPURIOUS r = {spurious_corr:.3f}', 
                     fontsize=12, fontweight='bold', color=PRIMARY_RED)

# Add warning text
axes[1, 1].text(0.5, 0.15, '⚠️ NOT real connectivity!\nJust volume conduction', 
                transform=axes[1, 1].transAxes, fontsize=11, ha='center',
                color=PRIMARY_RED, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='white', edgecolor=PRIMARY_RED))
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Visualization 9: Volume Conduction Creates Spurious Correlation', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"⚠️ Spurious envelope correlation: {spurious_corr:.3f}")
print("This is NOT true connectivity — both channels simply measure the same source!")
print("→ Solution: Use orthogonalization or source localization (covered in H01)")

---

## 9. Envelope Dynamics Across Frequency Bands

Different frequency bands have characteristic envelope dynamics:

| Band | Frequency | Typical Envelope Dynamics |
|------|-----------|---------------------------|
| **Theta** | 4-8 Hz | Slow, sustained fluctuations |
| **Alpha** | 8-13 Hz | Clear bursts, ~0.5-2s dynamics |
| **Beta** | 13-30 Hz | Faster fluctuations, ~0.2-0.5s |
| **Gamma** | 30-100 Hz | Very fast, often brief bursts |

These differences matter for connectivity analysis — the timescale of envelope
correlation depends on the frequency band being analyzed.

In [ ]:
# ============================================================================
# VISUALIZATION 10: Envelope Dynamics Across All Frequency Bands
# ============================================================================

# Generate a rich EEG-like signal with multiple components
np.random.seed(777)
fs_bands = 250
duration_bands = 8.0
t_bands = np.arange(0, duration_bands, 1/fs_bands)

# Create realistic multi-band signal
# Delta (slow background)
delta = 0.5 * np.sin(2 * np.pi * 2 * t_bands)

# Theta with slow modulation
theta_mod_bands = 0.5 + 0.5 * np.sin(2 * np.pi * 0.2 * t_bands)
theta_bands = theta_mod_bands * np.sin(2 * np.pi * 6 * t_bands)

# Alpha with burst pattern
alpha_mod_bands = 0.3 + 0.7 * np.exp(-((t_bands - 2) ** 2) / 0.8)
alpha_mod_bands += 0.5 * np.exp(-((t_bands - 5) ** 2) / 0.5)
alpha_mod_bands += 0.6 * np.exp(-((t_bands - 7) ** 2) / 0.4)
alpha_bands = alpha_mod_bands * np.sin(2 * np.pi * 10 * t_bands)

# Beta with faster fluctuations
beta_mod_bands = 0.4 + 0.4 * np.sin(2 * np.pi * 0.7 * t_bands)
beta_bands = beta_mod_bands * np.sin(2 * np.pi * 22 * t_bands)

# Gamma with brief bursts
gamma_mod_bands = 0.2 + 0.3 * np.exp(-((t_bands - 3) ** 2) / 0.1)
gamma_mod_bands += 0.25 * np.exp(-((t_bands - 6) ** 2) / 0.15)
gamma_bands = gamma_mod_bands * np.sin(2 * np.pi * 45 * t_bands)

# Combine
eeg_multiband = delta + theta_bands + alpha_bands + 0.5 * beta_bands + 0.3 * gamma_bands
eeg_multiband += 0.2 * np.random.randn(len(t_bands))

# Extract envelopes for each band
bands = {
    'Theta (4-8 Hz)': (4, 8),
    'Alpha (8-13 Hz)': (8, 13),
    'Beta (13-30 Hz)': (13, 30),
    'Gamma (30-60 Hz)': (30, 60)
}
band_colors = [COLORS["theta"], COLORS["alpha"], COLORS["beta"], COLORS["gamma"]]

# Create figure
fig, axes = plt.subplots(5, 1, figsize=(14, 14), sharex=True)

# Raw signal
axes[0].plot(t_bands, eeg_multiband, color='gray', linewidth=0.5, alpha=0.7)
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Raw Multi-Band EEG Signal', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Each band
for i, ((band_name, (low, high)), color) in enumerate(zip(bands.items(), band_colors)):
    env = extract_envelope(eeg_multiband, fs_bands, (low, high))
    env_norm = env / env.max()  # Normalize for comparison
    
    filt = bandpass_filter(eeg_multiband, low, high, fs_bands)
    filt_norm = filt / np.abs(filt).max()
    
    axes[i+1].plot(t_bands, filt_norm, color=color, linewidth=0.3, alpha=0.4)
    axes[i+1].plot(t_bands, env_norm, color=color, linewidth=2.5, label=band_name)
    axes[i+1].plot(t_bands, -env_norm, color=color, linewidth=2.5)
    axes[i+1].set_ylabel(f'{band_name.split()[0]}', fontsize=11)
    axes[i+1].set_title(f'{band_name}: {"Slow" if i < 2 else "Faster"} Envelope Dynamics', 
                        fontsize=12, fontweight='bold')
    axes[i+1].legend(loc='upper right')
    axes[i+1].grid(True, alpha=0.3)
    axes[i+1].set_ylim(-1.3, 1.3)

axes[4].set_xlabel('Time (s)', fontsize=12)

plt.suptitle('Visualization 10: Envelope Dynamics Differ Across Frequency Bands', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Observations:")
print("- Theta: Slow, sustained envelope changes")
print("- Alpha: Clear burst pattern (waxing and waning)")
print("- Beta: Faster envelope fluctuations")
print("- Gamma: Brief, transient bursts")

In [ ]:
# ============================================================================
# VISUALIZATION 11: Cross-Band Envelope Correlation Matrix
# ============================================================================

# Extract all envelopes
band_names = list(bands.keys())
envelopes = []
for band_name, (low, high) in bands.items():
    env = extract_envelope(eeg_multiband, fs_bands, (low, high))
    envelopes.append(env)

# Compute correlation matrix
n_bands = len(band_names)
corr_matrix = np.zeros((n_bands, n_bands))

for i in range(n_bands):
    for j in range(n_bands):
        corr_matrix[i, j] = np.corrcoef(envelopes[i], envelopes[j])[0, 1]

# Create figure
fig, ax = plt.subplots(figsize=(8, 7))

# Plot heatmap
im = ax.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Correlation', fontsize=12)

# Add labels
short_names = ['Theta', 'Alpha', 'Beta', 'Gamma']
ax.set_xticks(range(n_bands))
ax.set_yticks(range(n_bands))
ax.set_xticklabels(short_names, fontsize=11)
ax.set_yticklabels(short_names, fontsize=11)

# Add correlation values as text
for i in range(n_bands):
    for j in range(n_bands):
        color = 'white' if abs(corr_matrix[i, j]) > 0.5 else 'black'
        ax.text(j, i, f'{corr_matrix[i, j]:.2f}', ha='center', va='center', 
                fontsize=12, fontweight='bold', color=color)

ax.set_title('Cross-Band Envelope Correlation Matrix', fontsize=14, fontweight='bold')
ax.set_xlabel('Frequency Band', fontsize=12)
ax.set_ylabel('Frequency Band', fontsize=12)

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Diagonal = 1.0 (each band correlates perfectly with itself)")
print("- Off-diagonal values show cross-frequency amplitude coupling")
print("- Adjacent bands often show higher correlation")

---

## 10. Envelope Statistics

We can quantify envelope properties with various statistics:

| Statistic | Definition | Interpretation |
|-----------|------------|----------------|
| **Mean** | Average envelope | Overall oscillation strength |
| **Std** | Standard deviation | Variability of amplitude |
| **CV** | std/mean | Normalized variability |
| **Median** | 50th percentile | Robust central tendency |
| **P25, P75** | Quartiles | Distribution spread |

These statistics describe the oscillation's "behavior" over a recording.

In [ ]:
# ============================================================================
# FUNCTION 7: compute_envelope_statistics
# ============================================================================

def compute_envelope_statistics(
    envelope: NDArray[np.floating]
) -> dict:
    """
    Compute comprehensive statistics for an amplitude envelope.
    
    Parameters
    ----------
    envelope : NDArray[np.floating]
        Amplitude envelope time series.
    
    Returns
    -------
    dict
        Dictionary containing:
        - mean: Mean envelope value
        - std: Standard deviation
        - cv: Coefficient of variation (std/mean)
        - median: 50th percentile
        - p25: 25th percentile
        - p75: 75th percentile
        - iqr: Interquartile range (p75 - p25)
        - min: Minimum value
        - max: Maximum value
    """
    mean_val = np.mean(envelope)
    std_val = np.std(envelope)
    
    return {
        'mean': mean_val,
        'std': std_val,
        'cv': std_val / mean_val if mean_val > 0 else np.nan,
        'median': np.median(envelope),
        'p25': np.percentile(envelope, 25),
        'p75': np.percentile(envelope, 75),
        'iqr': np.percentile(envelope, 75) - np.percentile(envelope, 25),
        'min': np.min(envelope),
        'max': np.max(envelope)
    }

In [ ]:
# ============================================================================
# VISUALIZATION 12: Envelope Statistics Visualization
# ============================================================================

# Use alpha envelope from earlier
env_for_stats = extract_envelope(eeg_multiband, fs_bands, (8, 13))
stats = compute_envelope_statistics(env_for_stats)

# Create figure with GridSpec
fig = plt.figure(figsize=(14, 6))
gs = fig.add_gridspec(1, 4, width_ratios=[3, 0.8, 0.1, 1.2])

# Main time series plot
ax_main = fig.add_subplot(gs[0])
ax_main.plot(t_bands, env_for_stats, color=COLORS["alpha"], linewidth=1.5, alpha=0.8)

# Add horizontal lines for statistics
ax_main.axhline(y=stats['mean'], color=PRIMARY_RED, linestyle='-', linewidth=2, 
                label=f"Mean = {stats['mean']:.3f}")
ax_main.axhline(y=stats['median'], color=PRIMARY_GREEN, linestyle='--', linewidth=2,
                label=f"Median = {stats['median']:.3f}")

# Shade IQR region
ax_main.axhspan(stats['p25'], stats['p75'], color='gray', alpha=0.2, 
                label=f"IQR [{stats['p25']:.2f}, {stats['p75']:.2f}]")

ax_main.set_xlabel('Time (s)', fontsize=12)
ax_main.set_ylabel('Envelope Amplitude', fontsize=12)
ax_main.set_title('Alpha Envelope with Statistics', fontsize=12, fontweight='bold')
ax_main.legend(loc='upper right')
ax_main.grid(True, alpha=0.3)

# Histogram (rotated)
ax_hist = fig.add_subplot(gs[1], sharey=ax_main)
ax_hist.hist(env_for_stats, bins=30, orientation='horizontal', color=COLORS["alpha"], 
             alpha=0.7, edgecolor='white')
ax_hist.axhline(y=stats['mean'], color=PRIMARY_RED, linestyle='-', linewidth=2)
ax_hist.axhline(y=stats['median'], color=PRIMARY_GREEN, linestyle='--', linewidth=2)
ax_hist.axhspan(stats['p25'], stats['p75'], color='gray', alpha=0.2)
ax_hist.set_xlabel('Count', fontsize=11)
ax_hist.set_title('Distribution', fontsize=11, fontweight='bold')
plt.setp(ax_hist.get_yticklabels(), visible=False)

# Statistics table
ax_table = fig.add_subplot(gs[3])
ax_table.axis('off')

# Create table data
table_data = [
    ['Statistic', 'Value'],
    ['Mean', f"{stats['mean']:.4f}"],
    ['Std', f"{stats['std']:.4f}"],
    ['CV', f"{stats['cv']:.4f}"],
    ['Median', f"{stats['median']:.4f}"],
    ['P25', f"{stats['p25']:.4f}"],
    ['P75', f"{stats['p75']:.4f}"],
    ['IQR', f"{stats['iqr']:.4f}"],
    ['Min', f"{stats['min']:.4f}"],
    ['Max', f"{stats['max']:.4f}"]
]

table = ax_table.table(cellText=table_data, loc='center', cellLoc='center',
                       colWidths=[0.5, 0.5])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)

# Style header row
for i in range(2):
    table[(0, i)].set_facecolor('#E6E6E6')
    table[(0, i)].set_text_props(fontweight='bold')

ax_table.set_title('Envelope Statistics', fontsize=12, fontweight='bold', pad=20)

plt.suptitle('Visualization 12: Comprehensive Envelope Statistics', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Coefficient of Variation (CV) = {stats['cv']:.3f}")
print("→ CV indicates how variable the envelope is relative to its mean")

---

## 11. Practical Application — Alpha Blocking

**Alpha blocking** is a classic EEG phenomenon:
- When eyes are **closed**: Strong posterior alpha activity
- When eyes **open**: Alpha amplitude decreases dramatically

This demonstrates the utility of envelope analysis for detecting cognitive state changes.

In [ ]:
# ============================================================================
# VISUALIZATION 13: Alpha Blocking Demonstration
# ============================================================================

# Simulate alpha blocking: eyes closed → eyes open
np.random.seed(888)
fs_block = 250
duration_block = 10.0  # 5s eyes closed, 5s eyes open
t_block = np.arange(0, duration_block, 1/fs_block)
transition_time = 5.0  # seconds

# Create amplitude modulation (high before transition, low after)
alpha_amp = np.ones(len(t_block))
# Eyes closed: high alpha
alpha_amp[t_block < transition_time] = 1.0
# Eyes open: low alpha (blocking)
alpha_amp[t_block >= transition_time] = 0.25
# Smooth the transition
transition_samples = int(0.5 * fs_block)  # 0.5s transition
trans_idx = int(transition_time * fs_block)
alpha_amp[trans_idx:trans_idx + transition_samples] = np.linspace(1.0, 0.25, transition_samples)

# Generate alpha signal
alpha_block = alpha_amp * np.sin(2 * np.pi * 10 * t_block)
alpha_block += 0.15 * np.random.randn(len(t_block))

# Add some background noise
full_signal = alpha_block + 0.1 * np.random.randn(len(t_block))

# Extract alpha envelope
env_block = extract_envelope(full_signal, fs_block, (8, 13))
env_smooth = smooth_envelope_gaussian(env_block, sigma_samples=25)

# Compute statistics for each period
mask_closed = t_block < transition_time
mask_open = t_block >= transition_time
mean_closed = np.mean(env_smooth[mask_closed])
mean_open = np.mean(env_smooth[mask_open])

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Raw signal
axes[0].plot(t_block, full_signal, color='gray', linewidth=0.5, alpha=0.7)
axes[0].axvline(x=transition_time, color=PRIMARY_RED, linestyle='--', linewidth=2,
                label='Eyes open')
axes[0].axvspan(0, transition_time, color=PRIMARY_BLUE, alpha=0.1, label='Eyes closed')
axes[0].axvspan(transition_time, duration_block, color=PRIMARY_GREEN, alpha=0.1, 
                label='Eyes open')
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Raw EEG Signal: Alpha Blocking Paradigm', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Alpha-filtered with envelope
alpha_filt_block = bandpass_filter(full_signal, 8, 13, fs_block)
axes[1].plot(t_block, alpha_filt_block, color=COLORS["alpha"], linewidth=0.5, alpha=0.5)
axes[1].plot(t_block, env_smooth, color=PRIMARY_RED, linewidth=2.5, label='Envelope (smoothed)')
axes[1].plot(t_block, -env_smooth, color=PRIMARY_RED, linewidth=2.5)
axes[1].axvline(x=transition_time, color='black', linestyle='--', linewidth=2)
axes[1].axhline(y=mean_closed, color=PRIMARY_BLUE, linestyle=':', linewidth=2,
                label=f'Mean (closed) = {mean_closed:.3f}')
axes[1].axhline(y=mean_open, color=PRIMARY_GREEN, linestyle=':', linewidth=2,
                label=f'Mean (open) = {mean_open:.3f}')
axes[1].set_ylabel('Alpha Amplitude', fontsize=11)
axes[1].set_title('Alpha Envelope Shows Clear Blocking Effect', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Bar chart comparison
ax_bar = axes[2]
conditions = ['Eyes Closed', 'Eyes Open']
means = [mean_closed, mean_open]
colors_bar = [PRIMARY_BLUE, PRIMARY_GREEN]
bars = ax_bar.bar(conditions, means, color=colors_bar, edgecolor='black', linewidth=2)
ax_bar.set_ylabel('Mean Alpha Envelope', fontsize=11)
ax_bar.set_title('Alpha Blocking: Quantified Reduction', fontsize=12, fontweight='bold')

# Add values on bars
for bar, val in zip(bars, means):
    ax_bar.text(bar.get_x() + bar.get_width()/2, val + 0.02, f'{val:.3f}', 
                ha='center', fontweight='bold', fontsize=12)

# Add percent change
pct_change = 100 * (mean_open - mean_closed) / mean_closed
ax_bar.text(0.5, 0.6, f'Change: {pct_change:.1f}%', transform=ax_bar.transAxes,
            fontsize=14, ha='center', fontweight='bold', color=PRIMARY_RED)
ax_bar.grid(True, alpha=0.3, axis='y')

axes[2].set_xlabel('Time (s)', fontsize=12)

plt.suptitle('Visualization 13: Alpha Blocking — Classic EEG Phenomenon', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Mean envelope (eyes closed): {mean_closed:.4f}")
print(f"Mean envelope (eyes open):   {mean_open:.4f}")
print(f"Reduction: {pct_change:.1f}%")
print("\n→ Envelope analysis clearly quantifies the alpha blocking effect!")

---

## 12. Hyperscanning Application — Amplitude Coupling

In hyperscanning, we can analyze **envelope correlation between participants**:
- Do both brains show co-fluctuation of oscillatory amplitude?
- This may reflect shared arousal, attention, or cognitive state

**Key insight**: Amplitude coupling is a **different mechanism** than phase coupling.
Two participants might show:
- High PLV but low envelope correlation (phase sync, independent amplitudes)
- Low PLV but high envelope correlation (amplitude coupling, independent phases)
- Both high (full synchronization)
- Both low (no coupling)

In [ ]:
# ============================================================================
# VISUALIZATION 14: Hyperscanning — Amplitude Coupling Between Participants
# ============================================================================

# Simulate hyperscanning with amplitude coupling but INDEPENDENT phases
np.random.seed(999)
fs_hyper = 250
duration_hyper = 8.0
t_hyper = np.arange(0, duration_hyper, 1/fs_hyper)

# Shared envelope modulation (both participants attend together)
shared_env = 0.4 + 0.6 * np.sin(2 * np.pi * 0.3 * t_hyper)
shared_env += 0.4 * np.exp(-((t_hyper - 4) ** 2) / 0.8)  # Joint attention peak

# Participant A: alpha with shared envelope, random phase
phase_A = 2 * np.pi * 10 * t_hyper + np.random.uniform(0, 2*np.pi)
phase_A += 0.5 * np.random.randn(len(t_hyper)).cumsum() / fs_hyper  # Some drift
signal_A = shared_env * np.sin(phase_A) + 0.15 * np.random.randn(len(t_hyper))

# Participant B: alpha with shared envelope, INDEPENDENT random phase
phase_B = 2 * np.pi * 10 * t_hyper + np.random.uniform(0, 2*np.pi)
phase_B += 0.5 * np.random.randn(len(t_hyper)).cumsum() / fs_hyper  # Different drift
signal_B = shared_env * np.sin(phase_B) + 0.15 * np.random.randn(len(t_hyper))

# Extract envelopes
env_A = extract_envelope(signal_A, fs_hyper, (8, 13))
env_B = extract_envelope(signal_B, fs_hyper, (8, 13))

# Compute envelope correlation
env_corr_hyper = np.corrcoef(env_A, env_B)[0, 1]

# Compute PLV (should be low since phases are independent)
from src.phase import compute_plv_simple
filt_A = bandpass_filter(signal_A, 8, 13, fs_hyper)
filt_B = bandpass_filter(signal_B, 8, 13, fs_hyper)
phase_inst_A = np.angle(hilbert(filt_A))
phase_inst_B = np.angle(hilbert(filt_B))
plv_hyper = compute_plv_simple(phase_inst_A, phase_inst_B)

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top left: Both signals
axes[0, 0].plot(t_hyper, filt_A, color=PRIMARY_BLUE, linewidth=0.5, alpha=0.6, 
                label='Participant A')
axes[0, 0].plot(t_hyper, filt_B + 2, color=PRIMARY_RED, linewidth=0.5, alpha=0.6,
                label='Participant B (offset)')
axes[0, 0].axvspan(3.5, 4.5, color=ACCENT_GOLD, alpha=0.2, label='Joint attention')
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('Alpha Signals from Two Participants', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)

# Top right: Both envelopes overlaid
axes[0, 1].plot(t_hyper, env_A, color=PRIMARY_BLUE, linewidth=2, label='Envelope A')
axes[0, 1].plot(t_hyper, env_B, color=PRIMARY_RED, linewidth=2, label='Envelope B')
axes[0, 1].fill_between(t_hyper, env_A, env_B, color='gray', alpha=0.2)
axes[0, 1].axvspan(3.5, 4.5, color=ACCENT_GOLD, alpha=0.2)
axes[0, 1].set_ylabel('Envelope', fontsize=11)
axes[0, 1].set_title('Envelopes Co-Fluctuate (Amplitude Coupling)', 
                     fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
axes[0, 1].legend(loc='upper right')
axes[0, 1].grid(True, alpha=0.3)

# Bottom left: Scatter plot
axes[1, 0].scatter(env_A, env_B, color=ACCENT_PURPLE, alpha=0.3, s=20)
z = np.polyfit(env_A, env_B, 1)
p = np.poly1d(z)
env_sorted = np.sort(env_A)
axes[1, 0].plot(env_sorted, p(env_sorted), color='black', linewidth=2, linestyle='--')
axes[1, 0].set_xlabel('Envelope A', fontsize=11)
axes[1, 0].set_ylabel('Envelope B', fontsize=11)
axes[1, 0].set_title(f'Envelope Correlation: r = {env_corr_hyper:.3f}', 
                     fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Bottom right: Summary comparison
ax_sum = axes[1, 1]
metrics = ['PLV\n(Phase Coupling)', 'Envelope Corr\n(Amplitude Coupling)']
values = [plv_hyper, env_corr_hyper]
colors_sum = [ACCENT_PURPLE, PRIMARY_GREEN]
bars = ax_sum.bar(metrics, values, color=colors_sum, edgecolor='black', linewidth=2)
ax_sum.set_ylabel('Coupling Strength', fontsize=11)
ax_sum.set_ylim(0, 1)
ax_sum.set_title('Phase vs Amplitude Coupling', fontsize=12, fontweight='bold')

for bar, val in zip(bars, values):
    ax_sum.text(bar.get_x() + bar.get_width()/2, val + 0.03, f'{val:.3f}', 
                ha='center', fontweight='bold', fontsize=12)

ax_sum.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax_sum.grid(True, alpha=0.3, axis='y')

# Add interpretation
ax_sum.text(0.5, 0.75, 'High envelope corr\nLow PLV\n= Amplitude coupling only!', 
            transform=ax_sum.transAxes, fontsize=10, ha='center',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray'))

plt.suptitle('Visualization 14: Hyperscanning — Independent Phases, Coupled Amplitudes', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"PLV (phase coupling): {plv_hyper:.3f}")
print(f"Envelope correlation (amplitude coupling): {env_corr_hyper:.3f}")
print("\n→ These participants show AMPLITUDE coupling without PHASE coupling!")
print("→ This could reflect shared arousal/attention without neural synchronization")

In [ ]:
# ============================================================================
# VISUALIZATION 15: Three Coupling Scenarios Comparison
# ============================================================================

# Create three scenarios
np.random.seed(1111)

def create_scenario(phase_coupling, amplitude_coupling, t, fs):
    """Create a pair of signals with specified coupling types."""
    # Base envelope
    if amplitude_coupling:
        # Shared envelope
        shared_env = 0.5 + 0.5 * np.sin(2 * np.pi * 0.4 * t)
        env1, env2 = shared_env, shared_env
    else:
        # Independent envelopes
        env1 = 0.5 + 0.5 * np.sin(2 * np.pi * 0.4 * t)
        env2 = 0.5 + 0.5 * np.sin(2 * np.pi * 0.5 * t + np.pi)
    
    # Base phase
    if phase_coupling:
        # Same phase with small jitter
        base_phase = 2 * np.pi * 10 * t
        phase1 = base_phase
        phase2 = base_phase + 0.1 * np.random.randn(len(t)).cumsum() / fs
    else:
        # Independent phases
        phase1 = 2 * np.pi * 10 * t
        phase2 = 2 * np.pi * 10 * t + np.random.randn(len(t)).cumsum() * 0.5 / fs
    
    sig1 = env1 * np.sin(phase1) + 0.1 * np.random.randn(len(t))
    sig2 = env2 * np.sin(phase2) + 0.1 * np.random.randn(len(t))
    
    return sig1, sig2

fs_scen = 250
duration_scen = 5.0
t_scen = np.arange(0, duration_scen, 1/fs_scen)

# Scenario 1: Phase only
sig1_phase, sig2_phase = create_scenario(True, False, t_scen, fs_scen)
# Scenario 2: Amplitude only
sig1_amp, sig2_amp = create_scenario(False, True, t_scen, fs_scen)
# Scenario 3: Both
sig1_both, sig2_both = create_scenario(True, True, t_scen, fs_scen)

# Compute metrics for each
def compute_metrics(sig1, sig2, fs, band=(8, 13)):
    env1 = extract_envelope(sig1, fs, band)
    env2 = extract_envelope(sig2, fs, band)
    env_corr = np.corrcoef(env1, env2)[0, 1]
    
    filt1 = bandpass_filter(sig1, band[0], band[1], fs)
    filt2 = bandpass_filter(sig2, band[0], band[1], fs)
    ph1 = np.angle(hilbert(filt1))
    ph2 = np.angle(hilbert(filt2))
    plv = compute_plv_simple(ph1, ph2)
    
    return env1, env2, plv, env_corr

scenarios = [
    ("Phase Coupling Only", sig1_phase, sig2_phase),
    ("Amplitude Coupling Only", sig1_amp, sig2_amp),
    ("Both Phase & Amplitude", sig1_both, sig2_both)
]

# Create figure
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

for i, (title, sig1, sig2) in enumerate(scenarios):
    env1, env2, plv, env_corr = compute_metrics(sig1, sig2, fs_scen)
    
    # Left: envelopes
    axes[i, 0].plot(t_scen, env1, color=PRIMARY_BLUE, linewidth=2, label='Envelope 1')
    axes[i, 0].plot(t_scen, env2, color=PRIMARY_RED, linewidth=2, label='Envelope 2')
    axes[i, 0].set_ylabel('Envelope', fontsize=11)
    axes[i, 0].set_title(f'{title}', fontsize=12, fontweight='bold')
    axes[i, 0].legend(loc='upper right')
    axes[i, 0].grid(True, alpha=0.3)
    if i == 2:
        axes[i, 0].set_xlabel('Time (s)', fontsize=11)
    
    # Right: metrics bar
    metrics_vals = [plv, env_corr]
    metric_names = ['PLV', 'Env Corr']
    colors_m = [ACCENT_PURPLE, PRIMARY_GREEN]
    bars = axes[i, 1].bar(metric_names, metrics_vals, color=colors_m, edgecolor='black', linewidth=2)
    axes[i, 1].set_ylim(0, 1.1)
    axes[i, 1].set_ylabel('Value', fontsize=11)
    axes[i, 1].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    
    for bar, val in zip(bars, metrics_vals):
        axes[i, 1].text(bar.get_x() + bar.get_width()/2, val + 0.03, f'{val:.2f}', 
                        ha='center', fontweight='bold', fontsize=11)
    axes[i, 1].grid(True, alpha=0.3, axis='y')
    
    # Highlight the expected high values
    if i == 0:  # Phase only
        axes[i, 1].annotate('', xy=(0, plv), xytext=(0, plv + 0.15),
                           arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN, lw=2))
    elif i == 1:  # Amplitude only
        axes[i, 1].annotate('', xy=(1, env_corr), xytext=(1, env_corr + 0.15),
                           arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN, lw=2))

plt.suptitle('Visualization 15: Phase vs Amplitude Coupling — Three Scenarios', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Summary:")
print("- Scenario 1: High PLV, Low Env Corr → Phase coupling only")
print("- Scenario 2: Low PLV, High Env Corr → Amplitude coupling only")
print("- Scenario 3: High PLV, High Env Corr → Both mechanisms active")
print("\n→ These are INDEPENDENT connectivity mechanisms!")

---

## 13. Hands-On Exercises

### Exercise 1: Envelope Verification

Create an amplitude-modulated signal with a **known modulation function**.
Extract the envelope and verify it matches the original modulation.

1. Create a 3-second signal: 10 Hz carrier, 0.5 Hz sinusoidal modulation
2. Extract the envelope using `extract_envelope`
3. Compute correlation between extracted envelope and true modulation

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 1
# ==============================================

# Create AM signal with known modulation


# Extract envelope


# Compare with true modulation



<details>
<summary>💡 Click to see solution</summary>

```python
# Create AM signal
fs_ex1 = 250
duration_ex1 = 3.0
t_ex1 = np.arange(0, duration_ex1, 1/fs_ex1)

# Known modulation function
true_mod = 0.5 + 0.5 * np.sin(2 * np.pi * 0.5 * t_ex1)

# Create AM signal
carrier = np.sin(2 * np.pi * 10 * t_ex1)
am_signal_ex = true_mod * carrier

# Extract envelope
env_ex1 = extract_envelope(am_signal_ex, fs_ex1, (8, 13))

# Compute correlation
corr_ex1 = np.corrcoef(true_mod, env_ex1)[0, 1]

# Plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(t_ex1, true_mod, color=PRIMARY_GREEN, linewidth=2, linestyle='--', 
        label=f'True modulation')
ax.plot(t_ex1, env_ex1, color=PRIMARY_RED, linewidth=2, 
        label=f'Extracted envelope')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title(f'Envelope Recovery: Correlation = {corr_ex1:.4f}', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Correlation: {corr_ex1:.4f}")
print("→ The envelope accurately recovers the modulation function!")
```

</details>

---

### Exercise 2: Cross-Band Envelope Correlation

Generate an EEG-like signal with theta, alpha, and beta components.
Extract envelopes and compute cross-correlation between bands.

1. Create a 6-second multi-band signal
2. Extract envelopes for theta (4-8), alpha (8-13), beta (13-30)
3. Compute all pairwise correlations
4. Which bands have the most correlated amplitude dynamics?

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 2
# ==============================================

# Generate multi-band signal


# Extract envelopes for each band


# Compute pairwise correlations


# Visualize



<details>
<summary>💡 Click to see solution</summary>

```python
np.random.seed(222)
fs_ex2 = 250
t_ex2 = np.arange(0, 6, 1/fs_ex2)

# Create signal with some shared modulation between adjacent bands
theta_ex = (0.5 + 0.5*np.sin(2*np.pi*0.3*t_ex2)) * np.sin(2*np.pi*6*t_ex2)
alpha_ex = (0.4 + 0.6*np.sin(2*np.pi*0.3*t_ex2 + 0.2)) * np.sin(2*np.pi*10*t_ex2)  # Similar mod
beta_ex = (0.5 + 0.5*np.sin(2*np.pi*0.8*t_ex2)) * np.sin(2*np.pi*22*t_ex2)  # Different mod

signal_ex2 = theta_ex + alpha_ex + 0.5*beta_ex + 0.2*np.random.randn(len(t_ex2))

# Extract envelopes
env_theta_ex = extract_envelope(signal_ex2, fs_ex2, (4, 8))
env_alpha_ex = extract_envelope(signal_ex2, fs_ex2, (8, 13))
env_beta_ex = extract_envelope(signal_ex2, fs_ex2, (13, 30))

# Compute correlations
corr_theta_alpha = np.corrcoef(env_theta_ex, env_alpha_ex)[0, 1]
corr_theta_beta = np.corrcoef(env_theta_ex, env_beta_ex)[0, 1]
corr_alpha_beta = np.corrcoef(env_alpha_ex, env_beta_ex)[0, 1]

print("Cross-band envelope correlations:")
print(f"  Theta-Alpha: {corr_theta_alpha:.3f}")
print(f"  Theta-Beta:  {corr_theta_beta:.3f}")
print(f"  Alpha-Beta:  {corr_alpha_beta:.3f}")

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(t_ex2, env_theta_ex, label='Theta', color=COLORS["theta"])
axes[0].plot(t_ex2, env_alpha_ex, label='Alpha', color=COLORS["alpha"])
axes[0].plot(t_ex2, env_beta_ex, label='Beta', color=COLORS["beta"])
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Envelope')
axes[0].set_title('Band Envelopes', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Correlation matrix
corr_mat = np.array([
    [1.0, corr_theta_alpha, corr_theta_beta],
    [corr_theta_alpha, 1.0, corr_alpha_beta],
    [corr_theta_beta, corr_alpha_beta, 1.0]
])
im = axes[1].imshow(corr_mat, cmap='RdBu_r', vmin=-1, vmax=1)
axes[1].set_xticks([0, 1, 2])
axes[1].set_yticks([0, 1, 2])
axes[1].set_xticklabels(['Theta', 'Alpha', 'Beta'])
axes[1].set_yticklabels(['Theta', 'Alpha', 'Beta'])
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{corr_mat[i,j]:.2f}', ha='center', va='center', fontweight='bold')
axes[1].set_title('Cross-Band Correlation Matrix', fontweight='bold')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

print("\n→ Theta and Alpha have similar modulation → higher correlation")
```

</details>

### 💪 Exercise 3: Smoothing Trade-offs

Explore the trade-off between noise reduction and temporal smearing by comparing Gaussian smoothing with different sigma values (10, 50, 100 ms). Create a noisy envelope and show how larger sigma reduces noise but also smears sharp transients.

In [ ]:
# ==============================================
# EXERCISE 3: YOUR CODE HERE
# ==============================================
# Create a signal with sharp bursts
# Extract envelope
# Compare Gaussian smoothing with sigma = 10, 50, 100 ms
# Plot and analyze noise reduction vs temporal smearing
# ==============================================

<details>
<summary>💡 Click to see solution</summary>

```python
np.random.seed(333)
fs_ex3 = 500
t_ex3 = np.arange(0, 4, 1/fs_ex3)

# Create signal with sharp bursts (step-like envelope changes)
burst1 = np.zeros_like(t_ex3)
burst1[(t_ex3 >= 0.5) & (t_ex3 < 1.5)] = 1.0
burst2 = np.zeros_like(t_ex3)
burst2[(t_ex3 >= 2.5) & (t_ex3 < 3.2)] = 1.0

envelope_true = burst1 + 0.5*burst2 + 0.1
signal_ex3 = envelope_true * np.sin(2*np.pi*10*t_ex3) + 0.3*np.random.randn(len(t_ex3))

# Extract noisy envelope
env_ex3 = np.abs(hilbert(signal_ex3))

# Smooth with different sigmas
sigmas_ms = [10, 50, 100]
smoothed = {}
for sigma_ms in sigmas_ms:
    sigma_samples = int(sigma_ms * fs_ex3 / 1000)
    smoothed[sigma_ms] = gaussian_filter1d(env_ex3, sigma=sigma_samples)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original signal and true envelope
axes[0, 0].plot(t_ex3, signal_ex3, alpha=0.5, color=COLORS["signal"])
axes[0, 0].plot(t_ex3, envelope_true, 'k--', lw=2, label='True envelope')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Signal with Sharp Bursts', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Raw envelope
axes[0, 1].plot(t_ex3, env_ex3, alpha=0.7, color=COLORS["envelope"], label='Raw envelope')
axes[0, 1].plot(t_ex3, envelope_true, 'k--', lw=2, label='True envelope')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Envelope')
axes[0, 1].set_title('Raw Envelope (noisy)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Smoothed envelopes comparison
colors_sigma = [COLORS["theta"], COLORS["alpha"], COLORS["beta"]]
for idx, sigma_ms in enumerate(sigmas_ms):
    axes[1, 0].plot(t_ex3, smoothed[sigma_ms], label=f'σ={sigma_ms}ms', 
                    color=colors_sigma[idx], lw=2)
axes[1, 0].plot(t_ex3, envelope_true, 'k--', lw=2, label='True')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Smoothed Envelope')
axes[1, 0].set_title('Gaussian Smoothing Comparison', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xlim([0.3, 1.8])  # Zoom on first burst

# Compute metrics: noise and edge sharpness
noise_std = []
edge_response = []  # Time to reach 90% of max at burst onset
for sigma_ms in sigmas_ms:
    # Baseline noise (before first burst)
    baseline_idx = (t_ex3 >= 0.1) & (t_ex3 < 0.4)
    noise_std.append(np.std(smoothed[sigma_ms][baseline_idx]))
    
    # Edge response: measure slope at burst onset
    onset_idx = np.argmin(np.abs(t_ex3 - 0.5))
    slope = np.gradient(smoothed[sigma_ms][onset_idx:onset_idx+100]).max()
    edge_response.append(slope)

x_pos = np.arange(len(sigmas_ms))
width = 0.35
axes[1, 1].bar(x_pos - width/2, noise_std, width, label='Noise (std)', color=COLORS["theta"])
axes[1, 1].bar(x_pos + width/2, np.array(edge_response)*10, width, label='Edge response (×10)', 
               color=COLORS["alpha"])
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels([f'{s}ms' for s in sigmas_ms])
axes[1, 1].set_xlabel('Sigma')
axes[1, 1].set_ylabel('Value')
axes[1, 1].set_title('Trade-off: Noise vs Temporal Resolution', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("→ Larger sigma reduces noise but smears sharp transitions")
print("→ Choose sigma based on expected envelope dynamics in your data")
```

</details>

### 💪 Exercise 4: Envelope Correlation vs SNR

Investigate how noise affects envelope correlation measurements. Start with two signals that have perfectly correlated envelopes, then progressively add noise and measure how the correlation degrades.

In [ ]:
# ==============================================
# EXERCISE 4: YOUR CODE HERE
# ==============================================
# Create two signals with identical amplitude modulation
# Add different levels of noise (e.g., SNR = inf, 10, 5, 2, 1, 0.5 dB)
# Compute envelope correlation at each noise level
# Plot correlation as a function of SNR
# ==============================================

<details>
<summary>💡 Click to see solution</summary>

```python
np.random.seed(444)
fs_ex4 = 500
t_ex4 = np.arange(0, 10, 1/fs_ex4)

# Shared modulation envelope
shared_mod = 0.5 + 0.5*np.sin(2*np.pi*0.5*t_ex4)

# Clean signals with identical envelope
clean1 = shared_mod * np.sin(2*np.pi*10*t_ex4)
clean2 = shared_mod * np.sin(2*np.pi*10*t_ex4 + np.pi/4)  # Different phase

# SNR levels to test (in ratio, not dB for simplicity)
snr_levels = [np.inf, 20, 10, 5, 2, 1, 0.5, 0.2]
correlations = []

signal_power = np.var(clean1)

for snr in snr_levels:
    if snr == np.inf:
        noisy1, noisy2 = clean1, clean2
    else:
        noise_power = signal_power / snr
        noise1 = np.sqrt(noise_power) * np.random.randn(len(t_ex4))
        noise2 = np.sqrt(noise_power) * np.random.randn(len(t_ex4))
        noisy1 = clean1 + noise1
        noisy2 = clean2 + noise2
    
    # Extract envelopes
    env1 = np.abs(hilbert(noisy1))
    env2 = np.abs(hilbert(noisy2))
    
    # Compute correlation
    corr = np.corrcoef(env1, env2)[0, 1]
    correlations.append(corr)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# SNR vs Correlation curve
snr_labels = ['∞', '20', '10', '5', '2', '1', '0.5', '0.2']
axes[0].plot(range(len(snr_levels)), correlations, 'o-', color=COLORS["correlation"], 
             markersize=10, lw=2)
axes[0].axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Perfect correlation')
axes[0].set_xticks(range(len(snr_levels)))
axes[0].set_xticklabels(snr_labels)
axes[0].set_xlabel('SNR (ratio)')
axes[0].set_ylabel('Envelope Correlation')
axes[0].set_title('Envelope Correlation Degrades with Noise', fontweight='bold')
axes[0].set_ylim([0, 1.05])
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Show example at different SNR levels
example_snrs = [np.inf, 5, 0.5]
colors_ex = [COLORS["theta"], COLORS["alpha"], COLORS["beta"]]
for idx, snr in enumerate(example_snrs):
    if snr == np.inf:
        sig = clean1[:500]
        label = 'Clean (SNR=∞)'
    else:
        noise_power = signal_power / snr
        sig = clean1[:500] + np.sqrt(noise_power) * np.random.randn(500)
        label = f'SNR={snr}'
    axes[1].plot(t_ex4[:500], sig + idx*3, label=label, color=colors_ex[idx])

axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Amplitude (offset)')
axes[1].set_title('Signals at Different SNR Levels', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("→ Envelope correlation is sensitive to noise")
print("→ SNR < 2 significantly degrades correlation estimates")
print("→ Pre-filtering to the band of interest improves SNR")
```

</details>

### 💪 Exercise 5: Alpha Blocking Detection

Implement an automatic alpha blocking detector: given an EEG-like signal, detect periods where alpha power drops significantly (e.g., below 50% of baseline). Use the alpha envelope to identify "eyes open" periods.

In [ ]:
# ==============================================
# EXERCISE 5: YOUR CODE HERE
# ==============================================
# Create an EEG-like signal with alpha blocking periods
# Extract and smooth the alpha envelope
# Define baseline alpha power (e.g., first 2 seconds)
# Detect periods where alpha drops below threshold
# Visualize detected blocking periods
# ==============================================

<details>
<summary>💡 Click to see solution</summary>

```python
np.random.seed(555)
fs_ex5 = 250
t_ex5 = np.arange(0, 15, 1/fs_ex5)

# Create alpha envelope with blocking periods
alpha_mod = np.ones_like(t_ex5)
# Eyes open periods (alpha blocking)
blocking_periods = [(3, 5), (8, 10), (12, 14)]
for start, end in blocking_periods:
    # Gradual transition
    for i, ti in enumerate(t_ex5):
        if start - 0.2 < ti < start + 0.2:
            alpha_mod[i] = 1 - 0.7 * (1 + np.tanh((ti - start) * 10)) / 2
        elif start + 0.2 <= ti <= end - 0.2:
            alpha_mod[i] = 0.3
        elif end - 0.2 < ti < end + 0.2:
            alpha_mod[i] = 0.3 + 0.7 * (1 + np.tanh((ti - end) * 10)) / 2

# Generate EEG-like signal
alpha_comp = alpha_mod * np.sin(2*np.pi*10*t_ex5)
theta_comp = 0.3 * np.sin(2*np.pi*6*t_ex5)
beta_comp = 0.2 * np.sin(2*np.pi*20*t_ex5)
noise_ex5 = 0.15 * np.random.randn(len(t_ex5))

eeg_ex5 = alpha_comp + theta_comp + beta_comp + noise_ex5

# Extract and smooth alpha envelope
alpha_env = extract_envelope(eeg_ex5, fs_ex5, (8, 13))
alpha_env_smooth = smooth_envelope_gaussian(alpha_env, sigma_samples=int(0.2*fs_ex5))

# Compute baseline (first 2 seconds, assuming eyes closed)
baseline_mask = t_ex5 < 2
baseline_mean = np.mean(alpha_env_smooth[baseline_mask])
baseline_std = np.std(alpha_env_smooth[baseline_mask])

# Detection threshold: below 50% of baseline mean
threshold = 0.5 * baseline_mean

# Detect blocking periods
detected_blocking = alpha_env_smooth < threshold

# Find contiguous blocking segments
blocking_segments = []
in_block = False
start_idx = 0
for i, is_blocked in enumerate(detected_blocking):
    if is_blocked and not in_block:
        start_idx = i
        in_block = True
    elif not is_blocked and in_block:
        if i - start_idx > int(0.3*fs_ex5):  # Minimum duration 300ms
            blocking_segments.append((t_ex5[start_idx], t_ex5[i-1]))
        in_block = False
if in_block:
    blocking_segments.append((t_ex5[start_idx], t_ex5[-1]))

# Visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Raw EEG
axes[0].plot(t_ex5, eeg_ex5, color=COLORS["signal"], alpha=0.7)
axes[0].set_ylabel('EEG (µV)')
axes[0].set_title('Simulated EEG with Alpha Blocking', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Alpha envelope with threshold
axes[1].plot(t_ex5, alpha_env, alpha=0.4, color=COLORS["envelope"], label='Raw envelope')
axes[1].plot(t_ex5, alpha_env_smooth, color=COLORS["envelope"], lw=2, label='Smoothed')
axes[1].axhline(y=baseline_mean, color='green', linestyle='--', label=f'Baseline: {baseline_mean:.3f}')
axes[1].axhline(y=threshold, color='red', linestyle='--', label=f'Threshold (50%): {threshold:.3f}')
axes[1].fill_between(t_ex5, 0, alpha_env_smooth.max(), where=detected_blocking, 
                      alpha=0.3, color='red', label='Detected blocking')
axes[1].set_ylabel('Alpha Envelope')
axes[1].set_title('Alpha Envelope with Detection', fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Detection results
for start, end in blocking_periods:
    axes[2].axvspan(start, end, alpha=0.3, color='blue', label='True blocking' if start==3 else '')
for i, (start, end) in enumerate(blocking_segments):
    axes[2].axvspan(start, end, alpha=0.3, color='red', label='Detected' if i==0 else '')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Detection')
axes[2].set_title('True vs Detected Blocking Periods', fontweight='bold')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
axes[2].set_yticks([])

plt.tight_layout()
plt.show()

print(f"True blocking periods: {blocking_periods}")
print(f"Detected blocking periods: {blocking_segments}")
print(f"\n→ Envelope-based detection captures alpha blocking events")
print("→ Threshold and minimum duration can be tuned for sensitivity")
```

</details>

---

## 14. 📝 Summary

### Key Takeaways

1. **Amplitude Envelope Definition**
   - The envelope captures the slow-varying amplitude modulation of an oscillation
   - Extracted via Hilbert transform: `envelope = |analytic_signal|`
   - Represents the "strength" of neural oscillations over time

2. **Complete Extraction Pipeline**
   - Bandpass filter → Hilbert transform → Magnitude
   - Filter order affects transition sharpness (typically 4th order Butterworth)
   - Different bands reveal different aspects of neural dynamics

3. **Envelope is a Slow Signal**
   - Envelope frequencies are much lower than carrier frequencies
   - Typically < 2-3 Hz for most cognitive processes
   - Can be analyzed with its own power spectrum

4. **Smoothing Methods**
   - Moving average: Simple but may introduce discontinuities
   - Gaussian: Smooth output, sigma controls trade-off
   - Low-pass filter: Principled frequency cutoff
   - Trade-off: Noise reduction vs temporal smearing

5. **Envelope Correlation**
   - Measures amplitude coupling between signals or brain regions
   - Values: -1 (anti-correlated) to +1 (perfectly coupled)
   - Sensitive to volume conduction (can cause spurious correlations)

6. **Hyperscanning Applications**
   - Amplitude coupling indicates shared intensity patterns
   - Joint attention, emotional synchrony, motor coordination
   - Complementary to phase-based connectivity measures

### Functions Implemented

| Function | Purpose |
|----------|---------|
| `extract_envelope(signal, fs, band)` | Complete pipeline: filter + Hilbert + magnitude |
| `compute_envelope_psd(envelope, fs)` | Power spectrum of envelope |
| `smooth_envelope_moving_average()` | Simple sliding window |
| `smooth_envelope_gaussian()` | Gaussian filter smoothing |
| `smooth_envelope_lowpass()` | Low-pass Butterworth filter |
| `compute_envelope_correlation()` | Pearson correlation of envelopes |
| `compute_envelope_statistics()` | Mean, std, CV, median, percentiles |

---

## 15. 💬 Discussion Questions

1. **Phase vs Amplitude Coupling**: When would you prefer to use amplitude envelope correlation over phase-based measures (like PLV) for studying inter-brain synchrony? What types of cognitive processes might be better captured by each?

2. **Temporal Resolution Trade-off**: How does the choice of smoothing parameters (or filter cutoff) affect the interpretability of envelope dynamics? What considerations would guide your choice for studying rapid attentional shifts vs sustained emotional states?

3. **Volume Conduction Concerns**: We saw that volume conduction can create spurious envelope correlations. What strategies could help distinguish genuine amplitude coupling from volume conduction artifacts in hyperscanning studies?

4. **Cross-Frequency Coupling**: The cross-band envelope correlation we explored relates to cross-frequency coupling (CFC). How might amplitude-amplitude coupling between bands (e.g., theta-gamma) inform our understanding of inter-brain dynamics?

5. **Complementary Information**: In a hyperscanning study, you observe high PLV but low envelope correlation between two participants. What might this pattern indicate about their neural coordination? Conversely, what would low PLV but high envelope correlation suggest?

---

## 🔗 Next Steps

In the next notebook (**B04: Wavelets and Time-Frequency Analysis**), we will explore:
- Continuous wavelet transform for time-frequency decomposition
- Morlet wavelets and their properties
- Time-varying power and phase estimation
- Comparison with Hilbert-based methods

This will provide a complementary approach to extracting both amplitude and phase information with flexible time-frequency resolution.